In [1]:
import json
import requests
import pandas as pd

# Функция для извлечения данных

def extract_works_data(json_file_path):
    response = requests.get(json_file_path)
    data = response.content.decode('utf-8')
    data = json.loads(data)

    records = []  # сюда будем сохранять найденную информацию

    for group in data.get('groups', []):
        # Извлекаем год и название работы
        year = 'unknown'
        title = ''
        works = group.get('works', [])

        # Перебираем элементы списка works
        for work in works:
            # Ищем год издания (если есть в элементе списка, то забираем, нет - смотрим в следующем)
            if year == 'unknown':
                pub_date = work.get('publicationDate')
                if pub_date:
                    yr = pub_date.get('year')
                    if yr:
                        year = yr
            # Название работы - перебираем все названия и сохраняем самое длинное
            t = work.get('title', {})
            if isinstance(t, dict):
                t_val = t.get('value', '')
            else:
                t_val = str(t) if t else ''
            if len(t_val) > len(title):
                title = t_val
        # Ищем идентификаторы doi и eid из externalIdentifiers
        doi = None
        eid = None

        for ext_id in group.get('externalIdentifiers', []):
            # По элементам списка берем инормацию по типу и значению идентификатора
            id_type = ext_id.get('externalIdentifierType', {}).get('value', '').lower()
            id_value = ext_id.get('externalIdentifierId', {}).get('value')
            # Если doi, то сохраняем в doi, если eid - в eid, если ни то ни другое - не сохраняем
            if id_type == 'doi' and doi is None:
                doi = id_value
            elif id_type == 'eid' and eid is None:
                eid = id_value

        # Формируем массив
        records.append({
            'Год издания': year,
            'Название работы': title,
            'Идентификатор doi': doi,
            'Идентификатор eid': eid
        })

    # Сохраняем в датафрейм
    df = pd.DataFrame(records)

    #####
    # Проверяем, что данные выгружены корректно
    def validate_df(df, json_file_path):
      # выгружены все работы
      assert len(data['groups']) == len(df)
      # идентификаторы уникальны
      assert (len(df) - df['Идентификатор doi'].nunique() - df['Идентификатор doi'].isna().sum()) == 0
      assert (len(df) - df['Идентификатор eid'].nunique() - df['Идентификатор eid'].isna().sum()) == 0
      # идентификаторы начинаются с 10. для doi и 2 для eid
      assert (df['Идентификатор doi'].notna().sum() - df['Идентификатор doi'].str.startswith('10.').sum()) == 0
      assert (df['Идентификатор eid'].notna().sum() - df['Идентификатор eid'].str.startswith('2').sum()) == 0
      # нет пропусков в годе издания и названиях работ
      assert df['Год издания'].isna().sum() == 0
      assert df['Название работы'].isna().sum() == 0
      # годы издания соответствуют реальным
      years = df[df['Год издания'] != 'unknown']['Год издания'].astype(int)
      assert years.between(1994, 2026).all()
      print('Данные выгружены корректно!')
    #####

    # Применяем функцию проверки качества выгрузки
    validate_df(df, '/content/allWorks.json')
    print(f'\nКоличество работ ученого: {len(data['groups'])}')
    print(f'Выгружено работ: {len(records)}')

    # Возвращаем массив данных
    return pd.DataFrame(records)

# Применяем функцию
df = extract_works_data('https://orcid.org/0000-0002-8520-7267/allWorks.json?sort=date&sortAsc=false')
df['Год издания'] = df['Год издания'].astype('int')

# Считаем и выводим на экран работы без идентификаторов
no_identificator = 0
no_ind_list = []
for i in df.index:
  if df.loc[i, ['Идентификатор doi', 'Идентификатор eid']].isna().sum()==2:
    no_identificator+=1
    no_ind_list.append(i)
print(f'\n- Работ без идентификаторов doi и eid:{no_identificator}')
print(f'- Индексы работ без doi и eid:\n{no_ind_list}')

# Выводим массив данных
print('\nТаблица:')
display(df)

# Выгружаем в эксель
df.to_excel('список_работ.xlsx', index=False)

Данные выгружены корректно!

Количество работ ученого: 711
Выгружено работ: 711

- Работ без идентификаторов doi и eid:30
- Индексы работ без doi и eid:
[204, 205, 206, 207, 233, 235, 237, 238, 240, 242, 291, 292, 295, 380, 381, 382, 383, 385, 409, 426, 427, 428, 470, 500, 507, 517, 541, 542, 600, 621]

Таблица:


,Год издания,Название работы,Идентификатор doi,Идентификатор eid
0,2026,Deep Unfolding of Iterative Precoding-Based Nu...,10.1109/TCOMM.2025.3642692,None
1,2026,Resource Allocation Schemes for Scalable Panel...,10.1109/OJVT.2026.3652908,None
2,2025,On Deep Learning Hybrid Architectures for MIMO...,10.3390/electronics14234692,2-s2.0-105024537957
3,2025,Data-Oriented Channel Knowledge Map IoT Transm...,10.1109/VTC2025-Spring65109.2025.11174660,2-s2.0-105019047127
4,2025,Newmann Series-Based Precoding Weight Design f...,10.1109/VTC2025-Spring65109.2025.11174837,2-s2.0-105019051188
...,...,...,...,...
706,1996,Performance evaluation of OFDM transmission wi...,None,2-s2.0-0030362501
707,1996,Performance trade-offs with quasi-linearly amp...,None,2-s2.0-0029697021
708,1995,Adaptive serial OQAM-type receivers for mobile...,None,2-s2.0-0029191938
709,1994,Performance evaluation of MM-wave digital radi...,10.1109/SCVT.1994.574150,2-s2.0-85063366717
